# 구현한 방법론의 특징 및 성능

### 전처리의 특징
1. 특수문자 제거: '····…...↑→'와 같은 특수문자를 제거했다.
2. 괄호 안 정보 제거: (), <>, [] 안의 정보는 기사 내용과 크게 연관이 없는 정보이므로 제거했다.
3. 한자어는 한글로 변환했다.
4. 모든 기사의 헤드라인과 본문 내용을 토큰화하여 vocab으로 저장하되, 그 중 1음절이나 2회 미만출현단어는 제거했다.

### 사용한 형태소 분석기
Twitter를 사용했다. Twitter와 Mecab에 대해서 기타 조건을 같게 두고 성능을 비교했을 때, Twitter가 근소하게 더 좋은 정확도를 보였기 때문이다. Komoran은 Jpype와 konlpy 사이의 버전 충돌 문제로 인해 사용하지 못했다. 

||Mecab|Twitter|
|-------|-------------------------------|-------------------------------|
|vocab 개수|12659|13113|
|input_size|1621|1634|
|accuracy|12.8125|14.6875|

### 사용한 어휘들의 특징 및 vocab 개수
Twitter 형태소 분석기를 사용하여 명사만 추출했고, 총 vocab 개수는 13113개 였다.  
명사와 동사를 포함하여 vocab을 만들어보았으나, 정확도가 유의미하게 향상되지 않아 명사만 사용하는 것을 선택했다. 

### 하이퍼패러미터 값
activation function: ReLU  
batch size: 64 (train 데이터의 개수인 1280의 약수 중 가장 좋은 정확도를 보여서 선택했다)  
n_iters: 1280 (여러 번의 시도 끝에 가장 좋은 정확도를 보여서 선택했다)    
optimizer: RMSProp  

### 성능
위와 같이 하이퍼패러미터를 정했을 때, 19.0625 %의 정확도를 보였다.   
다음은 모든 조건을 위와 같이 했을 때, optimizer의 종류만을 바꾸며 정확도를 비교하여 표로 정리한 것이다. 

||SGD|SGD Nesterov|Adam|Adadelta|Adamax|Adagrad|RMSProp|
|---------|---------|---------|---------|---------|---------|---------|---------|
|accuracy|17.1875|17.5000|17.1875|16.8750|16.2500|16.8750|19.0625|

optimizer의 종류와 무관하게 전체적으로 정확도가 매우 낮은 것을 확인할 수 있다.    
따라서 더 정교한 모델을 사용하는 방향으로 성능을 높여봐야 한다고 생각된다. 

In [1]:
import re
import numpy as np
import collections
import hanja
import itertools
from konlpy.tag import Twitter
from numpy import repeat
from konlpy.tag import Mecab

import torch
import torch.nn as nn
import pandas as pd

In [2]:
# 원데이터를 가공해서 pd dataframe으로 읽기
news = pd.read_table("newsdata.tsv", sep = '\t')

In [3]:
## 전처리 및 파일을 FNN에 넣기 위한 필요한 모듈 설정

# 특수문자 제거하기
def remove_special_characters(line):
    text = re.sub('[····…...↑→]', '', line)
    
    return text


# () 혹은 <> 혹은 [] 안의 내용 제거하기
def remove_parentheses(line):
    pattern = re.compile(r'(\(|\<|\[).*(\)|\>|\])')
    text = re.sub(pattern, '', line)
    
    return text


# 한자어는 한글로 변환
def hanja_to_hangeul(line):
    text = hanja.translate(line, 'substitution')
    
    return text


'''
# 명사 추출하기 - komoran
def extract_nouns_komoran(line):
    komoran = Komoran()
    text = komoran.nouns(line)
    
    return text
'''


# 명사 추출하기 - twitter
def extract_nouns_twitter(line):
    twt = Twitter()
    nouns = twt.nouns(line)
    
    return nouns


# 명사 추출하기 - MeCab
def extract_nouns_mecab(line):
    mecab = Mecab()
    nouns = mecab.nouns(line)
    
    return nouns


# 명사와 동사 추출하기 - twitter
def extract_nouns_verbs_twitter(line):
    twt = Twitter()
    nouns_verbs = list()
    for word_pos in twt.pos(line):
        if word_pos[1] == 'Noun' or 'Verb':
            nouns_verbs.append(word_pos[0])
    
    return nouns_verbs

# 1음절 단어 모으기
def get_one_syllable_words(wordlist):
    one_syllable_words = []
    for word in wordlist:
        if len(word) == 1:
            one_syllable_words.append(word)
    
    return one_syllable_words

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    
    return text


# 기사 하나에 대해 전처리하기
def clean_article(article):
    # 특수문자 제거
    tokens = remove_special_characters(article)
    # 괄호 안의 내용 제거
    tokens = remove_parentheses(tokens)
    # 한자어는 한글로 변환
    tokens = hanja_to_hangeul(tokens)
    # 형태소 분석 통해 명사만 추출
    tokens = extract_nouns_twitter(tokens)
    # tokens = extract_nouns_verbs_twitter(tokens)
    # tokens = extract_nouns_mecab(tokens)
    # 길이 2 미만의 명사는 제외
    tokens = [w for w in tokens if len(w) > 1]

    return tokens


# 기사 하나에 대해 토큰화하여 vocab으로 저장
# inputs: 
#        article: string
#        vocab: collections.Counter
def add_article_to_vocab(vocab, article):
    # clean article
    tokens = clean_article(article)
    vocab.update(tokens)

    
# get article, clean and return a line of tokens
# inputs: 
#        article: string
#        vocab: a list of string 
def article_to_tokens(article, vocab):
    # clean article
    tokens = clean_article(article)
    # filter by vocab
    tokens = [w for w in tokens if w in vocab]
    return ' '.join(tokens)
    
    
# pandas dataframe 형식의 기사들을 받아 처리하기 (vocab 을 만드는 데 사용함)
# inputs: 
#        vocab: collections.Counter
#        df: pandas.dataframe 
def process_articles_df(vocab, df):
    for index, article in df.iterrows():
        add_article_to_vocab(vocab, article['headline'] + article['body'])
    
    
# list 형식의 기사들을 받아 처리하기 (개별 기사를 인코딩하는 데 사용함)
# inputs: 
#        vocab: a list of string
#        article_list: a list of string(article)    
def process_articles(vocab, article_list):
    lines = list()
    for article in article_list:
        line = article_to_tokens(article, vocab)
        lines.append(line)
    
    return lines
     
    
# token_list를 filename에 저장          
def save_list(token_list, filename):
    data = '\n'.join(token_list)
    file = open(filename,'w')
    file.write(data)
    file.close()

In [4]:
## 전처리 및 vocab 만들기

vocab_counter = collections.Counter()
process_articles_df(vocab_counter, news)
# keep tokens with >= 2 occurrence
min_occurance = 2
tokens = [k for k,c in vocab_counter.items() if c >= min_occurance]
# save tokens to a vocab file
save_list(tokens,'vocab.txt')
# print(len(tokens)) # 13113

/Users/juliejung/ml/env/lib/python3.6/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [4]:
## encoding, padding, one-hot encoding ...

# encoding
def texts_to_sequence(train_docs):
    # flatten train_docs for encoding
    flat_words = list(itertools.chain(*train_docs))
    flat_words = sorted(list(set(flat_words)))
    
    # encode char -> int, decode int-> char
    char_to_int = dict((c, i) for i, c in enumerate(flat_words))
    
    seq_out = list()
    for line in train_docs:
        encoded_seq = [char_to_int[char] / len(flat_words) for char in line]
        seq_out.append(encoded_seq)
    return seq_out


# padding
def pad_sequences(sequences, number, width):
    padd_seq_out = list()
    for line in sequences:
        line.extend(repeat(number, width - len(line)))
        padd_seq_out.append(line)
    return padd_seq_out
      
    
# one hot encoding
def one_hot_encoding(x, number_of_labels):
    output = np.zeros([np.size(x), number_of_labels])
    for i,index in enumerate(x):
        output[i,index]=1

    return output


# get the length of longest article out of 1600 articles
def get_max_width(docs):
    max_doc_length = len(docs[0])
    for doc in docs:
        if len(doc) > max_doc_length:
            max_doc_length = len(doc)
            
    return max_doc_length

In [5]:
# split new dataframe into train data and test data 
# x000 ~ x159 goes to train data
# x160 ~ x199 goes to test data

train_x = [] # a list of string
train_y = [] # a list of int

test_x = [] # a list of string
test_y = [] # a list of int

for index, article in news.iterrows():
    filename_number = int(article['filename'][1:4])
    article_text = article['headline'] + article['body']
    article_label = int(article['label'])
    if(filename_number in range(160, 200)):
        test_x.append(article_text)
        test_y.append(article_label)
    else:
        train_x.append(article_text)
        train_y.append(article_label)
        

In [6]:
## convert train_x, train_y, test_x, test_y into sequences

# load vocabulary
vocab_filename = 'vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)

# get the number of tokens for the longest article
train_x = process_articles(vocab, train_x) # len(train_x) should be 1280
test_x = process_articles(vocab, test_x)
max_article_length = get_max_width(train_x + test_x) # 1634
# print(max_article_length) # 1634

# encode, pad train_x and test_x
train_x = pad_sequences(texts_to_sequence(train_x), 0, max_article_length) # (1280, 1634)
test_x = pad_sequences(texts_to_sequence(test_x), 0, max_article_length)   # (320, 1634)

/Users/juliejung/ml/env/lib/python3.6/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [7]:
# Set seed

torch.manual_seed(0)

In [8]:
# convert train_x, train_y, test_x, test_y to torch.Tensor
train_x = torch.Tensor(train_x)
train_y = torch.Tensor(train_y)
train_y = train_y.type(torch.int64)

test_x = torch.Tensor(test_x)
test_y = torch.Tensor(test_y)
test_y = test_y.type(torch.int64)

# convert train_x, train_y to Tensor Dataset
train_dataset = torch.utils.data.TensorDataset(train_x, train_y)
test_dataset = torch.utils.data.TensorDataset(test_x, test_y)

# print(len(train_x), len(test_x)) # 1280 320

In [9]:
## MAKING DATASET ITERABLE

batch_size = 64
n_iters = 1280
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size = batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size = batch_size, shuffle=False) 

In [10]:
## Create Model Class

class FFN(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(FFN, self).__init__()
        # Linear function 1: 1634 --> 100
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        # Non-linearity 1
        self.relu1 = nn.ReLU()
        
        # Linear function 2: 100 --> 50
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        # Non-linearity 2
        self.relu2 = nn.ReLU()
        
        # Linear function 3 (readout) : 50 --> 10
        self.fc3 = nn.Linear(hidden_dim2, output_dim)  
        
    def forward(self, x):
        # Linear function 1
        out = self.fc1(x)
        # Non-linearity 1
        out = self.relu1(out)
        
        # Linear function 2
        out = self.fc2(out)
        # Non-linearity 2
        out = self.relu2(out)
        
        # Linear function 3 (readout)
        out = self.fc3(out)
        return out      

In [11]:
## INSTANTIATE MODEL CLASS

input_dim = 1634 # number of tokens in article with the most tokens
hidden1_dim = 100
hidden2_dim = 50
output_dim = 8

model = FFN(input_dim, hidden1_dim, hidden2_dim, output_dim)

#######################
#  USE GPU FOR MODEL  #
#######################

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

FFN(
  (fc1): Linear(in_features=1634, out_features=100, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=100, out_features=50, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=50, out_features=8, bias=True)
)

In [12]:
## INSTANTIATE LOSS CLASS

criterion = nn.CrossEntropyLoss()

In [13]:
## INSTANTIATE OPTIMIZER CLASS


# SGD
# learning_rate = 0.001
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) #17.1875

# Adam
# optimizer = torch.optim.Adam(model.parameters()) #17.1875

# Adadelta
# optimizer = torch.optim.Adadelta(model.parameters()) # 16.875

# Adamax
# optimizer = torch.optim.Adamax(model.parameters()) # 16.25

# RMSProp
optimizer = torch.optim.RMSprop(model.parameters()) # 19.0625

# Adagrad
# optimizer = torch.optim.Adagrad(model.parameters()) # 16.875

# SGD Nesterov 
# learning_rate = 0.01
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True) #17.5

In [ ]:
## TRAIN THE MODEL

iter = 0
for epoch in range(num_epochs):
    for i, (articles, labels) in enumerate(train_loader):

         #######################
        #  USE GPU FOR MODEL  #
        #######################
        articles = articles.view(-1, 1634).requires_grad_().to(device)
        labels = labels.to(device = device, dtype=torch.int64)
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model(articles)
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 100 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for articles, labels in test_loader:
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                articles = articles.view(-1, 1634).requires_grad_().to(device)

                # Forward pass only to get logits/output
                outputs = model(articles)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += labels.size(0)

                #######################
                #  USE GPU FOR MODEL  #
                #######################
                # Total correct predictions    
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()
                
            accuracy = 100 * float(correct) / float(total)
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 100. Loss: 1.8935710191726685. Accuracy: 15.3125
